### 7번 - 도로 속도표시 격자화

#### 한 격자 내의 다른 속도 표시가 있는 경우 사고 발생률에 영향이 있을 것이라 가정
#### 도로 속도표시판의 총 개수까지만 표기한 뒤 겹치는 속도 표지판은 걸러내고, 서로 다른 속도 표지판이 몇 개가 있는지, 그 속도는 무엇인지 격자화 진행

In [1]:
# 가져다 쓸 모듈들
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns # 예쁜 matplotlib
import folium # 지도 시각화
import json # geojson 파일용
import geopandas as gpd # geojson 파일용
import matplotlib.font_manager as fm # 한글 폰트 사용

/opt/app-root/lib/python3.6/site-packages/geopandas/_compat.py:91: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [2]:
filename = "30.대전광역시_차량등록현황_격자.geojson"
file = open(filename,encoding="utf8")
df2 = gpd.read_file(file)
df2

,gid,cars_cnt,geometry
0,다마846996,0,"MULTIPOLYGON (((127.32871 36.19342, 127.32871 ..."
1,다마847995,0,"MULTIPOLYGON (((127.32983 36.19252, 127.32982 ..."
2,다마847996,0,"MULTIPOLYGON (((127.32982 36.19342, 127.32982 ..."
3,다마847997,0,"MULTIPOLYGON (((127.32982 36.19432, 127.32982 ..."
4,다마847998,0,"MULTIPOLYGON (((127.32982 36.19522, 127.32982 ..."
...,...,...,...
54907,라바052223,0,"MULTIPOLYGON (((127.55799 36.39818, 127.55799 ..."
54908,라바052224,0,"MULTIPOLYGON (((127.55799 36.39908, 127.55799 ..."
54909,라바053222,0,"MULTIPOLYGON (((127.55910 36.39727, 127.55910 ..."
54910,라바053223,0,"MULTIPOLYGON (((127.55910 36.39818, 127.55910 ..."


In [3]:
filename = "3월22일제발마지막-3.json"
file = open(filename,encoding="utf8")
df = gpd.read_file(file)
df.drop(df.loc[df['index']==-1].index, inplace=True)
df=df.reset_index()
del df['level_0']
df

,gu,dong,jibun,char_ctt,index,geometry
0,서구,도안동,None,30,15040,POINT (127.34400 36.32066)
1,유성구,원신흥동,None,30,14519,POINT (127.34082 36.33462)
2,서구,둔산동,None,30,27290,POINT (127.39448 36.35359)
3,대덕구,읍내동,None,50,25260,POINT (127.38685 36.35398)
4,대덕구,오정동,None,30,32014,POINT (127.41365 36.35662)
...,...,...,...,...,...,...
6793,유성구,봉명동,None,50,14272,POINT (127.34024 36.34729)
6794,유성구,봉명동,None,50,14272,POINT (127.34020 36.34731)
6795,유성구,봉명동,None,50,12714,POINT (127.33297 36.33275)
6796,유성구,봉명동,None,50,12714,POINT (127.33304 36.33270)


In [4]:
df['char_ctt'].unique() # 이상치 발견

array(['30', '50', '40', '70', '60', '80', '70m'], dtype=object)

In [5]:
df[df['char_ctt']=='70m'] # 이상치

,gu,dong,jibun,char_ctt,index,geometry
5417,동구,가양동,None,70m,39870,POINT (127.44526 36.33893)


In [6]:
df.at[5417,'char_ctt']='70' # 이상치 수정

In [7]:
df['char_ctt'].unique() # 확인

array(['30', '50', '40', '70', '60', '80'], dtype=object)

In [8]:
df10=df.copy()
df10

,gu,dong,jibun,char_ctt,index,geometry
0,서구,도안동,None,30,15040,POINT (127.34400 36.32066)
1,유성구,원신흥동,None,30,14519,POINT (127.34082 36.33462)
2,서구,둔산동,None,30,27290,POINT (127.39448 36.35359)
3,대덕구,읍내동,None,50,25260,POINT (127.38685 36.35398)
4,대덕구,오정동,None,30,32014,POINT (127.41365 36.35662)
...,...,...,...,...,...,...
6793,유성구,봉명동,None,50,14272,POINT (127.34024 36.34729)
6794,유성구,봉명동,None,50,14272,POINT (127.34020 36.34731)
6795,유성구,봉명동,None,50,12714,POINT (127.33297 36.33275)
6796,유성구,봉명동,None,50,12714,POINT (127.33304 36.33270)


In [9]:
df10=df10.sort_values(by=['index','char_ctt'])
df10=df10.reset_index()
df10

,level_0,gu,dong,jibun,char_ctt,index,geometry
0,1764,유성구,송정동,None,70,267,POINT (127.25587 36.27973)
1,2763,유성구,송정동,None,70,267,POINT (127.25590 36.27967)
2,1749,유성구,세동,710답,80,595,POINT (127.26348 36.31406)
3,1750,유성구,세동,710답,80,595,POINT (127.26353 36.31399)
4,1747,유성구,세동,산83-13임,80,601,POINT (127.26334 36.31864)
...,...,...,...,...,...,...,...
6793,5246,서구,둔산동,None,50,54610,POINT (127.53698 36.40692)
6794,5247,대덕구,읍내동,None,50,54610,POINT (127.53703 36.40691)
6795,5240,대덕구,읍내동,None,50,54646,POINT (127.53838 36.41424)
6796,5241,서구,둔산동,None,50,54646,POINT (127.53832 36.41425)


### 속도 같은 경우 제거한 테이블

In [10]:
delete_list=[]
for i in range (1,len(df10['index'])):
    if df10['index'][i]==df10['index'][i-1] and df10['char_ctt'][i]==df10['char_ctt'][i-1]:
        print(i)
        delete_list.append(i)

1
3
5
7
9
11
13
15
17
19
23
24
26
28
29
32
40
41
44
46
48
49
54
56
57
58
61
62
63
70
71
72
74
75
76
78
79
84
85
88
89
92
93
95
98
101
102
103
105
106
108
111
115
116
117
119
120
122
123
124
125
127
128
130
133
135
136
138
139
140
141
143
144
146
148
150
152
153
154
156
158
160
161
166
169
171
173
176
179
180
181
184
185
187
192
193
196
199
200
201
204
207
209
211
213
215
217
218
221
223
225
229
231
232
234
235
237
239
240
242
245
247
248
250
251
252
254
256
257
258
259
262
263
266
269
270
271
272
275
278
283
284
286
288
289
292
296
298
299
300
302
304
305
307
308
311
312
314
315
317
320
322
327
329
331
333
335
337
340
343
344
348
349
351
352
353
354
357
360
364
366
368
369
371
372
375
377
378
380
381
383
386
387
389
390
393
394
397
398
400
401
404
406
407
409
410
412
413
415
416
417
419
420
421
422
423
426
427
428
430
433
434
436
437
439
443
444
446
447
448
449
451
453
454
455
456
457
459
460
461
462
464
466
468
470
472
474
476
477
479
481
482
483
485
487
489
491
492
493
495
497
498
50

In [11]:
df11=df10.drop(df10.loc[delete_list].index)

In [13]:
del df11['level_0']

In [14]:
df11=df11.reset_index()
df11

,level_0,gu,dong,jibun,char_ctt,index,geometry
0,0,유성구,송정동,None,70,267,POINT (127.25587 36.27973)
1,2,유성구,세동,710답,80,595,POINT (127.26348 36.31406)
2,4,유성구,세동,산83-13임,80,601,POINT (127.26334 36.31864)
3,6,유성구,송정동,None,70,612,POINT (127.26485 36.27547)
4,8,유성구,세동,산69-10임,80,708,POINT (127.26590 36.30715)
...,...,...,...,...,...,...,...
3016,6790,대덕구,읍내동,None,50,54414,POINT (127.53082 36.39466)
3017,6792,동구,주촌동,산23-7도,50,54547,POINT (127.53514 36.40105)
3018,6793,서구,둔산동,None,50,54610,POINT (127.53698 36.40692)
3019,6795,대덕구,읍내동,None,50,54646,POINT (127.53838 36.41424)


In [15]:
del df11['level_0']
df11

,gu,dong,jibun,char_ctt,index,geometry
0,유성구,송정동,None,70,267,POINT (127.25587 36.27973)
1,유성구,세동,710답,80,595,POINT (127.26348 36.31406)
2,유성구,세동,산83-13임,80,601,POINT (127.26334 36.31864)
3,유성구,송정동,None,70,612,POINT (127.26485 36.27547)
4,유성구,세동,산69-10임,80,708,POINT (127.26590 36.30715)
...,...,...,...,...,...,...
3016,대덕구,읍내동,None,50,54414,POINT (127.53082 36.39466)
3017,동구,주촌동,산23-7도,50,54547,POINT (127.53514 36.40105)
3018,서구,둔산동,None,50,54610,POINT (127.53698 36.40692)
3019,대덕구,읍내동,None,50,54646,POINT (127.53838 36.41424)


In [16]:
df3=df2.copy()
df3['속도표시 총 개수']=0
df3['속도표시 종류 수']=0
df3['속도표시-1']=0
df3['속도표시-2']=0
df3['속도표시-3']=0
df3

,gid,cars_cnt,geometry,속도표시 총 개수,속도표시 종류 수,속도표시-1,속도표시-2,속도표시-3
0,다마846996,0,"MULTIPOLYGON (((127.32871 36.19342, 127.32871 ...",0,0,0,0,0
1,다마847995,0,"MULTIPOLYGON (((127.32983 36.19252, 127.32982 ...",0,0,0,0,0
2,다마847996,0,"MULTIPOLYGON (((127.32982 36.19342, 127.32982 ...",0,0,0,0,0
3,다마847997,0,"MULTIPOLYGON (((127.32982 36.19432, 127.32982 ...",0,0,0,0,0
4,다마847998,0,"MULTIPOLYGON (((127.32982 36.19522, 127.32982 ...",0,0,0,0,0
...,...,...,...,...,...,...,...,...
54907,라바052223,0,"MULTIPOLYGON (((127.55799 36.39818, 127.55799 ...",0,0,0,0,0
54908,라바052224,0,"MULTIPOLYGON (((127.55799 36.39908, 127.55799 ...",0,0,0,0,0
54909,라바053222,0,"MULTIPOLYGON (((127.55910 36.39727, 127.55910 ...",0,0,0,0,0
54910,라바053223,0,"MULTIPOLYGON (((127.55910 36.39818, 127.55910 ...",0,0,0,0,0


In [17]:
for i in range (len(df['index'])):
    df3['속도표시 총 개수'][df['index'][i]]+=1

/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [18]:
for i in range (len(df11['index'])):
    df3['속도표시 종류 수'][df11['index'][i]]+=1

/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [19]:
checker=0
for i in range (1,len(df11['index'])):
    if df11['index'][i]!=df11['index'][i-1]:
        checker=0
    if df11['index'][i]==df11['index'][i-1]:
        checker+=1
    if checker==0:
        df3['속도표시-1'][df11['index'][i]]=df11['char_ctt'][i]
    elif checker==1:
        df3['속도표시-2'][df11['index'][i]]=df11['char_ctt'][i]
    elif checker==2:
        df3['속도표시-3'][df11['index'][i]]=df11['char_ctt'][i]

/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [20]:
df3['속도표시 총 개수'].unique()

array([ 0,  2,  4,  1,  3,  5,  6,  8,  7,  9, 10])

In [21]:
df3['속도표시 총 개수'].sum()

6798

In [22]:
len(df)

6798

In [23]:
df3['속도표시 종류 수'].unique()

array([0, 1, 2, 3])

In [24]:
df3['속도표시 종류 수'].sum()

3021

In [25]:
len(df11)

3021

In [26]:
checker=0
for i in range (1,len(df11['index'])):
    if df11['index'][i]!=df11['index'][i-1]:
        checker=0
    if df11['index'][i]==df11['index'][i-1]:
        checker+=1
    if checker==0:
        df3['속도표시-1'][df11['index'][i]]=df11['char_ctt'][i]
    elif checker==1:
        df3['속도표시-2'][df11['index'][i]]=df11['char_ctt'][i]
    elif checker==2:
        df3['속도표시-3'][df11['index'][i]]=df11['char_ctt'][i]

/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [27]:
df3

,gid,cars_cnt,geometry,속도표시 총 개수,속도표시 종류 수,속도표시-1,속도표시-2,속도표시-3
0,다마846996,0,"MULTIPOLYGON (((127.32871 36.19342, 127.32871 ...",0,0,0,0,0
1,다마847995,0,"MULTIPOLYGON (((127.32983 36.19252, 127.32982 ...",0,0,0,0,0
2,다마847996,0,"MULTIPOLYGON (((127.32982 36.19342, 127.32982 ...",0,0,0,0,0
3,다마847997,0,"MULTIPOLYGON (((127.32982 36.19432, 127.32982 ...",0,0,0,0,0
4,다마847998,0,"MULTIPOLYGON (((127.32982 36.19522, 127.32982 ...",0,0,0,0,0
...,...,...,...,...,...,...,...,...
54907,라바052223,0,"MULTIPOLYGON (((127.55799 36.39818, 127.55799 ...",0,0,0,0,0
54908,라바052224,0,"MULTIPOLYGON (((127.55799 36.39908, 127.55799 ...",0,0,0,0,0
54909,라바053222,0,"MULTIPOLYGON (((127.55910 36.39727, 127.55910 ...",0,0,0,0,0
54910,라바053223,0,"MULTIPOLYGON (((127.55910 36.39818, 127.55910 ...",0,0,0,0,0


In [28]:
df3[df3['속도표시 종류 수']==3]

,gid,cars_cnt,geometry,속도표시 총 개수,속도표시 종류 수,속도표시-1,속도표시-2,속도표시-3
17546,다바868220,6,"MULTIPOLYGON (((127.35280 36.39539, 127.35280 ...",8,3,30,50,60
30724,다바917239,0,"MULTIPOLYGON (((127.40742 36.41258, 127.40742 ...",4,3,30,40,50
42068,다바958118,8,"MULTIPOLYGON (((127.45322 36.30352, 127.45322 ...",4,3,30,40,50


#### 저장

In [29]:
df3.to_file("격자_07-도로속도표지.json", driver="GeoJSON")